# Example Notebook and Installation Check
This notebook provides an example of how to run an experiment with the `locomotion` package. It also acts as a check to make sure your package has been installed correctly.

---

## Import Check
First, we make sure that we are able to import the locomotion package.

In [ ]:
import sys
import os
import numpy as np
PATH_TO_DIRECTORY = os.getcwd()

# If this works, then locomotion has been installed into your system.
import locomotion

## Data Setup
Next, we setup the data for our experiment, which involves writing `.json` files that captures the experimental data for each dataset. This can be done manually, as we've done below, or by using `scripts/json_generator.ipynb`. 

Note that 1 `.json` file needs to be generated for each experimental group, with a JSON item for each dataset in the group. In our example, we have 2 experimental groups with 2 datasets each, thus we create 2 `.json` files, each containing 2 JSON items.

In [ ]:
# No need to touch this
species = "medaka"
exp_type = "SS"
x_lims = (0, 300)
y_lims = (0, 200)
frames_per_sec = 20
unit_conversion = 1.6
start_time = 0
end_time = 120
baseline_start_time = 0
baseline_end_time = 60

import json
json_items_ss = [{
    "name": "SS_01",
    "data_file_location": PATH_TO_DIRECTORY + "/../samples/sample_data/SS_01.dat",
    "animal_attributes":
    {
      "species": species,
      "exp_type": exp_type,
      "ID": "01"
    },
    "capture_attributes": 
    {
      "x_lims": x_lims,
      "y_lims": y_lims,
      "input_unit" : "px",
      "output_unit" : "mm",
      "input_per_output_unit": unit_conversion,
      "frames_per_sec": frames_per_sec,
      "start_time": start_time,
      "end_time": end_time,
      "baseline_start_time": baseline_start_time,
      "baseline_end_time": baseline_end_time
    },
    "additional_info": 
    {
      "point": [100, 100]
    }
  },
{
    "name": "SS_02",
    "data_file_location": PATH_TO_DIRECTORY + "/../samples/sample_data/SS_02.dat",
    "animal_attributes":
    {
      "species": species,
      "exp_type": exp_type,
      "ID": "02"
    },
    "capture_attributes": 
    {
      "x_lims": x_lims,
      "y_lims": y_lims,
      "input_unit" : "px",
      "output_unit" : "mm",
      "input_per_output_unit": unit_conversion,
      "frames_per_sec": frames_per_sec,
      "start_time": start_time,
      "end_time": end_time,
      "baseline_start_time": baseline_start_time,
      "baseline_end_time": baseline_end_time
    },
    "additional_info": 
    {
      "point": [50, 50]
    }
  }]
json_items_nss = [{
    "name": "NSS_01",
    "data_file_location": PATH_TO_DIRECTORY + "/../samples/sample_data/NSS_01.dat",
    "animal_attributes":
    {
      "species": species,
      "exp_type": exp_type,
      "ID": "01"
    },
    "capture_attributes": 
    {
      "x_lims": x_lims,
      "y_lims": y_lims,
      "input_unit" : "px",
      "output_unit" : "mm",
      "input_per_output_unit": unit_conversion,
      "frames_per_sec": frames_per_sec,
      "start_time": start_time,
      "end_time": end_time,
      "baseline_start_time": baseline_start_time,
      "baseline_end_time": baseline_end_time
    },
    "additional_info": 
    {
      "point": [30, 30]
    }
  },
{
    "name": "NSS_02",
    "data_file_location": PATH_TO_DIRECTORY + "/../samples/sample_data/NSS_02.dat",
    "animal_attributes":
    {
      "species": species,
      "exp_type": exp_type,
      "ID": "02"
    },
    "capture_attributes": 
    {
      "x_lims": x_lims,
      "y_lims": y_lims,
      "input_unit" : "px",
      "output_unit" : "mm",
      "input_per_output_unit": unit_conversion,
      "frames_per_sec": frames_per_sec,
      "start_time": start_time,
      "end_time": end_time,
      "baseline_start_time": baseline_start_time,
      "baseline_end_time": baseline_end_time
    },
    "additional_info": 
    {
      "point": [10, 10]
    }
  }]

outfile_ss = PATH_TO_DIRECTORY + "/../samples/sample_check_SS.json"
outfile_nss = PATH_TO_DIRECTORY + "/../samples/sample_check_NSS.json"
jsonstr_ss = json.dumps(json_items_ss, indent=4)
jsonstr_nss = json.dumps(json_items_nss, indent=4)
with open(outfile_ss, "w") as outfile:
    outfile.write(jsonstr_ss)
    print("SS information wrote to %s" % outfile_ss)
with open(outfile_nss, "w") as outfile:
    outfile.write(jsonstr_nss)
    print("NSS information wrote to %s" % outfile_nss)

## Setup Animal Objects
This section will create the animal objects using the provided `.json` files, and will begin smoothening the curved data needed to run the methods in `trajectory.py`. 

Additionally, this section will show how to check the smoothened paths to ensure that smoothening parameters produces the intended results.

In [ ]:
info_files = [outfile_ss, outfile_nss]
animals = locomotion.setup_animal_objs( info_files )
for a in animals:
    # Smoothen curve data and add to animal object
    mse = locomotion.trajectory.populate_curve_data(
        a, 
        smooth_order=3, 
        smooth_window=25, 
        smooth_method='savgol'
    )
    print(f"MSE of smoothening for {a.get_name()} is {mse}.")
    
    #Plot smoothened path to check result
    locomotion.write.plot_path(a, 'results/')
    locomotion.write.plot_path(a, 'results/', 'smoothed_path', 'smooth_X', 'smooth_Y')
    

## Prepare Animal Objects for BDD Calculations

This section demonstrates the methods used to populate variables in the animal objects, as well as how to define normalization methods for these variables.

In [ ]:
# Populate other variables needed for calculation
for a in animals:
    first_deriv, velocity = locomotion.trajectory.populate_velocity( a )
    locomotion.trajectory.populate_curvature(a, first_deriv=first_deriv, velocity=velocity)
    locomotion.trajectory.populate_distance_from_point(a, "point", 'Dist to Point', col_names=['smooth_X', 'smooth_Y'])

In [ ]:
# Define standard normalization method using the mean and standard 
# deviation of variables across all animal objects
raw_vals = {'Velocity':[], 'Curvature':[]}

# Extract all values from all animals
for a in animals:
    for var in ['Velocity', 'Curvature']:
        raw_vals[var].extend(a.get_vals(var))

# Calculate mean and std and use it to define new normalization method
for var in ['Velocity', 'Curvature']:
    mean = np.mean(raw_vals[var])
    std = np.std(raw_vals[var])
    for a in animals:
        a.add_norm_standard(var, 'universal', mean, std)

In [ ]:
# Define bounded normalization method using predefined lower and 
# upper bounds for Dist to Point
lower_bound = 0
upper_bound = 100
for a in animals:
    a.add_norm_bounded('Dist to Point', 'bounded', lower_bound, upper_bound)

## Trajectory Methods
This section will run through the 2 main methods provided in `trajectory.py`:
* `trajectory.compute_all_bdd()` computes the pair-wise Behavioral Distortion Distance (BDD) between all animals in the experiment,
* `trajectory.compute_all_iibdd()` computes the Intra-Individual BDD of each animal in the experiment, by computing the BDD of an animal to itself over a pair of non-overlapping time intervals of equal length.

In [ ]:
# Testing BDD methods
variables = ['Velocity','Curvature', 'Dist to Point']
norm_mode = ['universal', 'universal', 'bounded']
start_time, end_time = 0, 60
distances = locomotion.trajectory.compute_all_bdd(animals,
                                                variables,
                                                start_time,
                                                end_time,
                                                norm_mode)


In [ ]:
output_directory, outfile_name = PATH_TO_DIRECTORY + "/results", "check_BDD"
try: # Safety check to ensure that data folder exists, and makes it otherwise.
    os.mkdir(output_directory)
except FileExistsError:
    pass
sort_table, square_table = False, False
color_min, color_max = 0.1, 0.2
locomotion.write.post_process(animals,
                             distances,
                             output_directory,
                             outfile_name,
                             sort_table,
                             square_table,
                             color_min,
                             color_max)

In [ ]:
print(distances)
# Print result should be similar to the following:
# [['', 0.12754357411745945, 0.12055945264105619, 0.13650692307401976], 
# ['', '', 0.10431838786760918, 0.11668354968697327], 
# ['', '', '', 0.06529442683095017], 
# ['', '', '', '']]

In [ ]:
# Testing IIBDD Methods
variables = ['Velocity','Curvature']
norm_mode = 'spec'
num_samples = 5
iibdds = locomotion.trajectory.compute_all_iibdd(animals, variables, norm_mode, num_samples)

In [ ]:
output_directory, outfile_name = PATH_TO_DIRECTORY + "/results", "check_IIBDD"
try: # Safety check to ensure that data folder exists, and makes it otherwise.
    os.mkdir(output_directory)
except FileExistsError:
    pass
locomotion.write.write_iibdd_to_csv(animals, iibdds, output_directory, outfile_name)

## Heatmap Methods
This section will run through the main method provided in `heatmap.py`, namely `heatmap.compute_all_csd()`, which calculates the pairwise Conformal Spatiotemporal Distance (CSD) between all animals in the experiment. 

In [ ]:
# Setup animal objects for CSD comparison
start_time, end_time = 0, 120
for a in animals:
    locomotion.heatmap.populate_surface_data(a, 
                                             start_time=start_time, 
                                             end_time=end_time)

In [ ]:
distances = locomotion.heatmap.compute_all_csd(animals)

In [ ]:
output_directory, outfile_name = PATH_TO_DIRECTORY + "/results", "check_CSD"
sort_table, square_table = False, False
color_min, color_max = 0, 0.02
locomotion.write.post_process(animals,
                             distances,
                             output_directory,
                             outfile_name,
                             sort_table,
                             square_table,
                             color_min,
                             color_max)

# Once you are done, please remember to clear all outputs with 'Cell > All Output > Clear'.